This notebook requests the access token to the Spotify API, collects and saves raw data in the `.json` format.

In [1]:
# Import necessary packages

from requests import post,get
import json
import numpy as np
import pandas as pd

Token endpoint URI and the format of the request is based on the [information page](https://developer.spotify.com/documentation/web-api/tutorials/getting-started#create-an-app) of Spotify.

In [2]:
with open("../.env", "r") as f:
    credentials = json.load(f)


url = "https://accounts.spotify.com/api/token"

headers = {"Content-Type": "application/x-www-form-urlencoded"}

data = {
    "grant_type": "client_credentials",
    "client_id": credentials["CLIENT_ID"],  
    "client_secret": credentials["CLIENT_SECRET"]
}

# Make the POST request
response = post(url, headers=headers, data=data)

# Check the response status and print the token if successful
if response.status_code == 200:
    token_info = response.json()
    access_token = token_info['access_token']
    print(token_info)
else:
    print("Failed to obtain token:", response.status_code)
    print(response.json())
    

{'access_token': 'BQDy0--aBjMFJ8otF54Fl_dB2x1aHf8yQHCkowdsPsiP7XUUgWNziKcweDXlUfTvIKVU4rJ8tUEkpCRJNBtnG5rJPDjvoY1tpGN3DGp4lgWwjlSbYag', 'token_type': 'Bearer', 'expires_in': 3600}


Having obtained the access token valid for 1 hour, we can start data collection. First, we will collect IDs of all playlists from a categoty in the Netherlands

In [10]:
header = {"Authorization": "Bearer " + access_token}

# Pop playlists were selected as they are most popular
category = "pop"

def get_playlists_links(num_of_playlists = 100):

    playlist_ids = []
    # maximum number of items to return is set to 50 by Spotify
    one_request_max = 50

    def extract_links(response):
        if response.status_code == 200:
            _data_page = response.json()

            # TEMPORARY FOR DEBUGGING
            with open("../data/raw/response.json", "w") as f:
                json.dump(_data_page, f)
            _df = pd.DataFrame(_data_page["playlists"]["items"])
            
            return _df["href"].tolist()
        else:
            print("Error: " + str(response.status_code))
            print(response.json())
            


    for num in range(num_of_playlists // one_request_max):
        response = get(f"https://api.spotify.com/v1/browse/categories/{category}/playlists?offset={50*num}&limit=50", headers=header)
        playlist_ids.extend(extract_links(response))

    # collect remaining playlists with smaller limit
    if num_of_playlists % one_request_max != 0:
        response = get(f"https://api.spotify.com/v1/browse/categories/{category}/playlists?offset={50 * (num_of_playlists // one_request_max)}%limit={num_of_playlists % one_request_max}", headers=header)
        playlist_ids.extend(extract_links(response))

    return playlist_ids
    

In [12]:
playlist_urls = get_playlists_links()

KeyError: 'href'

In [ ]:

len(set(playlist_urls))

22